In [2]:
!pip show langgraph


Name: langgraph
Version: 0.0.50
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /home/mark/Zynamis/gig_chatbot_backend/venv/lib/python3.10/site-packages
Requires: langchain-core, uuid6
Required-by: 


In [3]:
# --- LangGraph compatibility imports ---
try:
    # v0.2+ has START
    from langgraph.graph import StateGraph, START, END
    _USE_START = True
except Exception:
    # v0.1.x (no START)
    from langgraph.graph import StateGraph, END
    START = None
    _USE_START = False

from typing import TypedDict, Optional

# --- State type ---
class ChatState(TypedDict):
    user_input: str
    intent: Optional[str]

# --- Provide a minimal start node (used only on old versions) ---
def _start_node(state: ChatState) -> ChatState:
    print("▶️ start:", state)
    return state

# --- Your router + intent logic ---
def router_node(state: ChatState) -> ChatState:
    print("🔀 router received:", state)
    return state

def route_from_intent(state: ChatState) -> str:
    intent = (state.get("intent") or "").strip()
    allowed = {"register_event", "rsvp_help", "analytics_guide", "gig_intro"}
    return intent if intent in allowed else "fallback"

# If you already defined these in your project, you can remove these stubs.
def register_event(state: ChatState) -> ChatState:
    print("📌 register_event"); return state
def rsvp_help(state: ChatState) -> ChatState:
    print("📌 rsvp_help"); return state
def analytics_guide(state: ChatState) -> ChatState:
    print("📌 analytics_guide"); return state
def gig_intro(state: ChatState) -> ChatState:
    print("📌 gig_intro"); return state
def fallback(state: ChatState) -> ChatState:
    print("⚠️ fallback"); return state

def build_graph_compat() -> "Pregel":  # runtime returns a compiled graph
    builder = StateGraph(ChatState)

    # Nodes
    builder.add_node("router", router_node)
    builder.add_node("register_event", register_event)
    builder.add_node("rsvp_help", rsvp_help)
    builder.add_node("analytics_guide", analytics_guide)
    builder.add_node("gig_intro", gig_intro)
    builder.add_node("fallback", fallback)

    # Entry wiring differs by version
    if _USE_START:
        # v0.2+: explicit START edge is required
        builder.add_edge(START, "router")
    else:
        # v0.1.x: use a start node + set_entry_point
        builder.add_node("start", _start_node)
        builder.add_edge("start", "router")
        builder.set_entry_point("start")

    # Conditional routing from router
    builder.add_conditional_edges(
        "router",
        route_from_intent,
        {
            "register_event": "register_event",
            "rsvp_help": "rsvp_help",
            "analytics_guide": "analytics_guide",
            "gig_intro": "gig_intro",
            "fallback": "fallback",
        },
    )

    # All leaves must terminate at END
    for node in ["register_event", "rsvp_help", "analytics_guide", "gig_intro", "fallback"]:
        builder.add_edge(node, END)

    return builder.compile()

gig_chatbot_graph = build_graph_compat()
print("✅ Graph compiled. Using START entry:", _USE_START)


✅ Graph compiled. Using START entry: True


/home/mark/Zynamis/gig_chatbot_backend/venv/lib/python3.10/site-packages/langgraph/graph/graph.py:30: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langgraph.pregel import Channel, Pregel


In [4]:
print("---- Test 1: known intent ----")
state = {"user_input": "I want to sign up", "intent": "register_event"}
for event in gig_chatbot_graph.stream(state):
    print("➡️", event)

print("\n---- Test 2: unknown intent (fallback) ----")
state = {"user_input": "what is this?", "intent": "unknown"}
for event in gig_chatbot_graph.stream(state):
    print("➡️", event)


---- Test 1: known intent ----


KeyError: '__start__'